<a href="https://colab.research.google.com/github/TaliPaoHH/TelecomX-parte1/blob/main/analisis_churn_telecomx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis Completo de Churn - Telecom X
**Proyecto integral de Data Science para identificar factores de cancelación**


In [ ]:
# === Configuración inicial e imports ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
from datetime import datetime
import warnings
from scipy import stats
from sklearn.preprocessing import StandardScaler, LabelEncoder
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('default')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("ANÁLISIS COMPLETO DE CHURN - TELECOM X")
print("="*60)
print("Proyecto de Data Science para retención de clientes")
print("="*60)


## 1. Extracción de datos desde API


In [ ]:
def cargar_datos_api():
    """
    Función para cargar datos desde la API de GitHub con manejo robusto de errores
    """
    try:
        url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"
        print("Conectando a la API...")
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()
        df = pd.DataFrame(data)
        print("Datos cargados exitosamente")
        print(f"Dimensiones: {df.shape[0]} filas x {df.shape[1]} columnas")
        return df
    except requests.exceptions.RequestException as e:
        print(f"Error de conexión: {e}")
        return None
    except Exception as e:
        print(f"Error inesperado: {e}")
        return None

df_raw = cargar_datos_api()


## 2. Exploración inicial y comprensión del dataset


In [ ]:
if df_raw is not None:
    print("\nPASO 2: EXPLORACIÓN INICIAL DEL DATASET")
    print("-" * 50)

    def explorar_estructura_datos(df):
        """
        Función para explorar la estructura básica del dataset
        """
        print("INFORMACIÓN GENERAL DEL DATASET:")
        print("=" * 40)

        print(f"Número de filas: {df.shape[0]:,}")
        print(f"Número de columnas: {df.shape[1]}")
        try:
            mem_mb = df.memory_usage(deep=True).sum() / 1024**2
        except Exception:
            mem_mb = df.memory_usage().sum() / 1024**2
        print(f"Tamaño en memoria: {mem_mb:.2f} MB")

        print("\nTIPOS DE DATOS POR COLUMNA:")
        print("=" * 40)

        tipo_datos = pd.DataFrame({
            'Columna': df.columns,
            'Tipo_Datos': df.dtypes.astype(str),
            'Valores_Nulos': df.isnull().sum(),
            'Porcentaje_Nulos': (df.isnull().sum() / len(df) * 100).round(2),
            'Valores_Unicos': df.nunique(),
            'Ejemplo_Valor': [df[col].dropna().iloc[0] if not df[col].dropna().empty else 'N/A' for col in df.columns]
        })

        print(tipo_datos.to_string(index=False))

        numericas = df.select_dtypes(include=[np.number]).columns.tolist()
        categoricas = df.select_dtypes(include=['object', 'category']).columns.tolist()

        print(f"\nVariables numéricas ({len(numericas)}): {numericas}")
        print(f"Variables categóricas ({len(categoricas)}): {categoricas}")

        return tipo_datos, numericas, categoricas

    tipo_datos_info, cols_numericas, cols_categoricas = explorar_estructura_datos(df_raw)


## 3. Identificación y manejo de inconsistencias


In [ ]:
def identificar_inconsistencias(df, cols_numericas=None, cols_categoricas=None):
    """
    Función para identificar y reportar inconsistencias en los datos
    """
    print("ANÁLISIS DE CALIDAD DE DATOS:")
    print("=" * 35)

    valores_nulos = df.isnull().sum()
    if valores_nulos.sum() > 0:
        print("Valores nulos encontrados:")
        for col, count in valores_nulos[valores_nulos > 0].items():
            print(f"  - {col}: {count} ({count/len(df)*100:.2f}%)")
    else:
        print("No hay valores nulos")

    duplicados = df.duplicated().sum()
    print(f"\nFilas duplicadas: {duplicados}")

    if cols_categoricas:
        print("\nANÁLISIS DE VARIABLES CATEGÓRICAS:")
        for col in cols_categoricas:
            valores_unicos = df[col].dropna().unique()
            print(f"\n  - {col}:")
            print(f"    * Valores únicos: {len(valores_unicos)}")
            if len(valores_unicos) <= 10:
                print(f"    * Valores: {list(valores_unicos)}")
            else:
                print(f"    * Top 5: {list(df[col].value_counts().head().index)}")

    if cols_numericas:
        print("\nANÁLISIS DE VARIABLES NUMÉRICAS:")
        for col in cols_numericas:
            col_data = df[col].dropna()
            if len(col_data) == 0:
                continue
            print(f"\n  - {col}:")
            print(f"    * Rango: {col_data.min()} - {col_data.max()}")
            print(f"    * Media: {col_data.mean():.2f}")
            print(f"    * Valores negativos: {(col_data < 0).sum()}")
            print(f"    * Valores cero: {(col_data == 0).sum()}")

    return valores_nulos, duplicados

nulos_info, duplicados_info = identificar_inconsistencias(df_raw, cols_numericas, cols_categoricas)


## 4. Limpieza y transformación de datos


In [ ]:
def limpiar_y_transformar_datos(df, cols_categoricas):
    """
    Función completa para limpiar y transformar los datos
    """
    df_clean = df.copy()
    print("Aplicando transformaciones...")

    dups = df_clean.duplicated().sum()
    if dups > 0:
        df_clean = df_clean.drop_duplicates()
        print(f"Eliminados {dups} registros duplicados")

    for col in df_clean.columns:
        if df_clean[col].isnull().sum() > 0:
            if str(df_clean[col].dtype) in ['object', 'category']:
                moda = df_clean[col].mode()
                moda = moda.iloc[0] if not moda.empty else 'Unknown'
                df_clean[col] = df_clean[col].fillna(moda)
            else:
                mediana = df_clean[col].median()
                df_clean[col] = df_clean[col].fillna(mediana)

    print("Estandarizando variables categóricas binarias (si aplica)...")
    binary_map = {
        'Yes': 1, 'No': 0,
        'Si': 1, 'Sí': 1, 'No': 0,
        'TRUE': 1, 'FALSE': 0,
        'True': 1, 'False': 0,
        'Male': 1, 'Female': 0,
        'Masculino': 1, 'Femenino': 0,
        'M': 1, 'F': 0
    }

    for col in cols_categoricas:
        vals = df_clean[col].dropna().unique()
        if len(vals) == 2:
            df_clean[col] = df_clean[col].replace(binary_map)

    churn_col = None
    for col in df_clean.columns:
        low = col.lower()
        if 'churn' in low or 'evasion' in low or 'cancel' in low:
            churn_col = col
            break

    return df_clean, churn_col

df_clean, churn_column = limpiar_y_transformar_datos(df_raw, cols_categoricas)


## 5. Creación de columna: Cuentas_Diarias


In [ ]:
def crear_cuentas_diarias(df):
    """Crea 'Cuentas_Diarias' a partir de una columna mensual si existe."""
    keywords = ['monthly', 'mensual', 'total', 'charge', 'cargo', 'bill', 'factura']
    facturation_cols = [col for col in df.columns if any(k in col.lower() for k in keywords)]
    print(f"Columnas de facturación identificadas: {facturation_cols}")
    if facturation_cols:
        monthly_col = facturation_cols[0]
        df[monthly_col] = pd.to_numeric(df[monthly_col], errors='coerce')
        df['Cuentas_Diarias'] = (df[monthly_col] / 30.0).fillna(0)
        print(f"Columna 'Cuentas_Diarias' creada basada en {monthly_col}")
        print("Estadísticas Cuentas_Diarias:")
        print(f"  - Media: ${df['Cuentas_Diarias'].mean():.2f}")
        print(f"  - Mediana: ${df['Cuentas_Diarias'].median():.2f}")
        print(f"  - Min-Max: ${df['Cuentas_Diarias'].min():.2f} - ${df['Cuentas_Diarias'].max():.2f}")
    else:
        print("No se encontraron columnas de facturación para calcular cuentas diarias")
    return df

df_clean = crear_cuentas_diarias(df_clean)


## 6. Análisis descriptivo


In [ ]:
def analisis_descriptivo_completo(df, churn_col, cols_numericas, cols_categoricas):
    print("ESTADÍSTICAS DESCRIPTIVAS GENERALES:")
    print("=" * 45)
    if cols_numericas:
        desc_stats = df[cols_numericas].describe().round(2)
        print(desc_stats)
    else:
        desc_stats = pd.DataFrame()
        print("No hay columnas numéricas para describir.")

    if churn_col and churn_col in df.columns:
        print(f"\nANÁLISIS ESPECÍFICO DE CHURN ({churn_col}):")
        print("=" * 45)
        churn_dist = df[churn_col].value_counts(dropna=False)
        churn_pct = df[churn_col].value_counts(normalize=True, dropna=False) * 100
        print("Distribución de Churn:")
        for val in churn_dist.index:
            cantidad = churn_dist.loc[val]
            porcentaje = churn_pct.loc[val]
            print(f"  - {val}: {cantidad:,} clientes ({porcentaje:.2f}%)")
        try:
            if pd.api.types.is_numeric_dtype(df[churn_col]):
                tasa_churn = pd.to_numeric(df[churn_col], errors='coerce').mean()
            else:
                vals = df[churn_col].dropna().unique()
                if len(vals) == 2:
                    yes_alias = {'yes', 'sí', 'si', '1', 'true'}
                    val_churn = None
                    for v in vals:
                        if str(v).strip().lower() in yes_alias:
                            val_churn = v
                            break
                    if val_churn is None:
                        val_churn = vals[0]
                    tasa_churn = (df[churn_col] == val_churn).mean()
                else:
                    tasa_churn = np.nan
        except Exception:
            tasa_churn = np.nan
        print(f"\nTasa de Churn: {tasa_churn:.2%}" if pd.notna(tasa_churn) else "\nTasa de Churn: no determinable")

        print("\nANÁLISIS POR VARIABLES CATEGÓRICAS:")
        for col in cols_categoricas:
            if col != churn_col and df[col].nunique() <= 10:
                print(f"\n  - Churn por {col}:")
                try:
                    crosstab = pd.crosstab(df[col], df[churn_col], normalize='index') * 100
                    for idx in crosstab.index:
                        churn_rate_segment = crosstab.loc[idx].max()
                        print(f"    * {idx}: {churn_rate_segment:.1f}% de churn (máx. por categoría)")
                except Exception as e:
                    print(f"    * No se pudo calcular: {e}")
    return desc_stats

estadisticas_desc = analisis_descriptivo_completo(df_clean, churn_column, cols_numericas, cols_categoricas)


## 7. Visualizaciones avanzadas (matplotlib / seaborn)


In [ ]:
def crear_visualizaciones_avanzadas(df, churn_col, cols_numericas, cols_categoricas):
    fig = plt.figure(figsize=(20, 16))
    # 1. Distribución de Churn
    plt.subplot(3, 3, 1)
    if churn_col and churn_col in df.columns:
        churn_counts = df[churn_col].value_counts(dropna=False)
        labels = [str(x) for x in churn_counts.index]
        plt.pie(churn_counts.values, labels=labels, autopct='%1.1f%%', startangle=90)
        plt.title(f'Distribución de {churn_col}', fontweight='bold')
    else:
        plt.text(0.5, 0.5, 'Columna de Churn no identificada', ha='center', va='center')
        plt.title('Distribución de Churn')
    # 2. Histograma de la primera variable numérica
    plt.subplot(3, 3, 2)
    if len(cols_numericas) > 0:
        primera_numerica = cols_numericas[0]
        try:
            plt.hist(pd.to_numeric(df[primera_numerica], errors='coerce').dropna(), bins=30, alpha=0.7, edgecolor='black')
            plt.title(f'Distribución de {primera_numerica}')
            plt.xlabel(primera_numerica)
            plt.ylabel('Frecuencia')
        except Exception as e:
            plt.text(0.5, 0.5, f'No disponible: {e}', ha='center')
    else:
        plt.text(0.5, 0.5, 'Sin variables numéricas', ha='center')
    # 3. Boxplot comparativo si hay churn
    plt.subplot(3, 3, 3)
    if churn_col and len(cols_numericas) > 0 and churn_col in df.columns:
        try:
            sns.boxplot(data=df, x=churn_col, y=cols_numericas[0])
            plt.title(f'{cols_numericas[0]} por {churn_col}')
        except Exception as e:
            plt.text(0.5, 0.5, f'No disponible: {e}', ha='center')
    # 4. Matriz de correlación
    plt.subplot(3, 3, 4)
    num_cols = [c for c in cols_numericas if pd.api.types.is_numeric_dtype(df[c])]
    if len(num_cols) >= 2:
        try:
            corr_matrix = df[num_cols].corr(numeric_only=True)
            sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0, square=True, cbar_kws={'shrink': 0.8})
            plt.title('Matriz de Correlación')
        except Exception as e:
            plt.text(0.5, 0.5, f'No disponible: {e}', ha='center')
    else:
        plt.text(0.5, 0.5, 'Variables numéricas insuficientes', ha='center')
    # 5. Churn por primera categórica
    plt.subplot(3, 3, 5)
    if churn_col and len(cols_categoricas) > 0:
        primera_cat = cols_categoricas[0] if cols_categoricas[0] != churn_col else (cols_categoricas[1] if len(cols_categoricas) > 1 else None)
        if primera_cat is not None and churn_col in df.columns:
            try:
                pd.crosstab(df[primera_cat], df[churn_col]).plot(kind='bar', stacked=True, ax=plt.gca())
                plt.title(f'Churn por {primera_cat}')
                plt.xticks(rotation=45)
            except Exception as e:
                plt.text(0.5, 0.5, f'No disponible: {e}', ha='center')
        else:
            plt.text(0.5, 0.5, 'Categóricas insuficientes', ha='center')
    # 6. Distribución de Cuentas_Diarias
    plt.subplot(3, 3, 6)
    if 'Cuentas_Diarias' in df.columns:
        try:
            plt.hist(pd.to_numeric(df['Cuentas_Diarias'], errors='coerce').dropna(), bins=30, alpha=0.7, edgecolor='black')
            plt.title('Distribución de Cuentas Diarias')
            plt.xlabel('Cuentas Diarias ($)')
            plt.ylabel('Frecuencia')
        except Exception as e:
            plt.text(0.5, 0.5, f'No disponible: {e}', ha='center')
    else:
        plt.text(0.5, 0.5, 'Sin Cuentas_Diarias', ha='center')
    # 7. Tasa de churn por rangos de cuentas
    plt.subplot(3, 3, 7)
    if 'Cuentas_Diarias' in df.columns and churn_col and churn_col in df.columns:
        try:
            tmp = df.copy()
            tmp['Rango_Cuentas'] = pd.cut(pd.to_numeric(tmp['Cuentas_Diarias'], errors='coerce'), bins=5, labels=['Muy Bajo', 'Bajo', 'Medio', 'Alto', 'Muy Alto'])
            pd.crosstab(tmp['Rango_Cuentas'], tmp[churn_col], normalize='index').plot(kind='bar', ax=plt.gca())
            plt.title('Tasa de Churn por Rango de Cuentas')
            plt.xticks(rotation=45)
        except Exception as e:
            plt.text(0.5, 0.5, f'No disponible: {e}', ha='center')
    else:
        plt.text(0.5, 0.5, 'Requiere Churn y Cuentas_Diarias', ha='center')
    # 8. Top categorías
    plt.subplot(3, 3, 8)
    if len(cols_categoricas) > 0:
        cat_col = cols_categoricas[0] if (churn_col is None or cols_categoricas[0] != churn_col) else (cols_categoricas[1] if len(cols_categoricas) > 1 else None)
        if cat_col:
            try:
                top_cats = df[cat_col].value_counts().head(8)
                top_cats.plot(kind='barh', ax=plt.gca())
                plt.title(f'Top Categorías - {cat_col}')
            except Exception as e:
                plt.text(0.5, 0.5, f'No disponible: {e}', ha='center')
        else:
            plt.text(0.5, 0.5, 'Categóricas insuficientes', ha='center')
    # 9. Evolución temporal
    plt.subplot(3, 3, 9)
    time_cols = [col for col in df.columns if any(k in col.lower() for k in ['date', 'time', 'mes', 'fecha'])]
    if time_cols:
        time_col = time_cols[0]
        try:
            if not np.issubdtype(df[time_col].dtype, np.datetime64):
                df[time_col] = pd.to_datetime(df[time_col], errors='coerce')
            ts = df.dropna(subset=[time_col]).copy()
            ts = ts.sort_values(time_col)
            if churn_col and churn_col in df.columns and pd.api.types.is_numeric_dtype(df[churn_col]):
                time_churn = ts.groupby(time_col)[churn_col].mean()
                time_churn.plot(kind='line', marker='o', ax=plt.gca())
                plt.title(f'Evolución de Churn por {time_col}')
            else:
                ts[time_col].value_counts().sort_index().plot(kind='line', marker='o', ax=plt.gca())
                plt.title(f'Distribución temporal - {time_col}')
        except Exception as e:
            plt.text(0.5, 0.5, f'No disponible: {e}', ha='center')
    else:
        plt.text(0.5, 0.5, 'No hay columnas temporales', ha='center')
        plt.title('Análisis Temporal')
    plt.tight_layout()
    plt.suptitle('ANÁLISIS VISUAL COMPLETO - TELECOM X', fontsize=16, fontweight='bold', y=1.02)
    plt.show()
    return fig

_ = crear_visualizaciones_avanzadas(df_clean, churn_column, cols_numericas, cols_categoricas)


## 8. Análisis de correlación avanzado


In [ ]:
def analisis_correlacion_avanzado(df, churn_col, cols_numericas):
    print("ANÁLISIS DE CORRELACIONES:")
    print("=" * 35)
    correlaciones_fuertes = []
    num_cols = [c for c in cols_numericas if pd.api.types.is_numeric_dtype(df[c])]
    if len(num_cols) >= 2:
        corr_matrix = df[num_cols].corr(numeric_only=True)
        for i in range(len(corr_matrix.columns)):
            for j in range(i+1, len(corr_matrix.columns)):
                var1, var2 = corr_matrix.columns[i], corr_matrix.columns[j]
                corr_val = corr_matrix.iloc[i, j]
                if pd.notna(corr_val) and abs(corr_val) > 0.3:
                    correlaciones_fuertes.append((var1, var2, float(corr_val)))
        correlaciones_fuertes.sort(key=lambda x: abs(x[2]), reverse=True)
        print("Top correlaciones (|r| > 0.3):")
        for i, (var1, var2, corr) in enumerate(correlaciones_fuertes[:10], 1):
            print(f"  {i}. {var1} ↔ {var2}: {corr:.3f}")
    if churn_col and churn_col in df.columns and num_cols:
        try:
            if pd.api.types.is_numeric_dtype(df[churn_col]):
                churn_correlations = df[num_cols].corrwith(df[churn_col]).sort_values(key=lambda s: s.abs(), ascending=False)
                print("\nVariables más correlacionadas con churn:")
                for var, corr in churn_correlations.head(10).items():
                    if var != churn_col and pd.notna(corr):
                        print(f"  - {var}: {corr:.3f}")
        except Exception as e:
            print(f"No fue posible calcular correlación con churn: {e}")
    return correlaciones_fuertes

correlaciones_info = analisis_correlacion_avanzado(df_clean, churn_column, cols_numericas)


## 9. Insights y conclusiones avanzadas


In [ ]:
def generar_insights_avanzados(df, churn_col, cols_categoricas):
    insights = []
    completeness = (1 - df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100
    insights.append(f"Calidad de datos: {completeness:.1f}% completo, {len(df)} clientes analizados")
    churn_rate = np.nan
    if churn_col and churn_col in df.columns:
        try:
            if pd.api.types.is_numeric_dtype(df[churn_col]):
                churn_rate = df[churn_col].mean()
            else:
                vals = df[churn_col].dropna().unique()
                if len(vals) == 2:
                    yes_alias = {'yes', 'sí', 'si', '1', 'true'}
                    val_churn = None
                    for v in vals:
                        if str(v).strip().lower() in yes_alias:
                            val_churn = v
                            break
                    if val_churn is None:
                        val_churn = vals[0]
                    churn_rate = (df[churn_col] == val_churn).mean()
        except Exception:
            churn_rate = np.nan
    if pd.notna(churn_rate):
        sever = "CRÍTICA" if churn_rate > 0.3 else ("ALTA" if churn_rate > 0.2 else "MODERADA")
        insights.append(f"Tasa de churn actual: {churn_rate:.1%} - {sever}")
    gender_cols = [col for col in cols_categoricas if any(word in col.lower() for word in ['gender', 'genero', 'sexo'])]
    if gender_cols and churn_col and churn_col in df.columns and pd.api.types.is_numeric_dtype(df[churn_col]):
        gender_col = gender_cols[0]
        try:
            gender_churn = df.groupby(gender_col)[churn_col].mean()
            if len(gender_churn) >= 2:
                diff = float(gender_churn.max() - gender_churn.min())
                if diff > 0.05:
                    max_gender = str(gender_churn.idxmax())
                    insights.append(f"Diferencia por género: {max_gender} presenta {diff:.1%} más churn que el otro grupo")
        except Exception:
            pass
    if 'Cuentas_Diarias' in df.columns and churn_col and churn_col in df.columns and pd.api.types.is_numeric_dtype(df[churn_col]):
        try:
            churn_daily = df.loc[df[churn_col] == 1, 'Cuentas_Diarias'].mean()
            no_churn_daily = df.loc[df[churn_col] == 0, 'Cuentas_Diarias'].mean()
            diff_daily = churn_daily - no_churn_daily
            if pd.notna(diff_daily):
                insights.append(f"Clientes que cancelan gastan ${abs(diff_daily):.2f} {'menos' if diff_daily < 0 else 'más'} diariamente")
        except Exception:
            pass
    num_count = len(df.select_dtypes(include=[np.number]).columns)
    if num_count >= 3:
        insights.append(f"Dataset con {num_count} variables numéricas útiles para análisis predictivo")
    if churn_col and churn_col in df.columns and 'Cuentas_Diarias' in df.columns and pd.api.types.is_numeric_dtype(df[churn_col]):
        total_churn_customers = int((df[churn_col] == 1).sum())
        avg_revenue_lost = float(df.loc[df[churn_col] == 1, 'Cuentas_Diarias'].mean() * 365)
        if not np.isnan(avg_revenue_lost):
            total_revenue_at_risk = total_churn_customers * avg_revenue_lost
            insights.append(f"Ingresos anuales en riesgo (estimado): ${total_revenue_at_risk:,.0f}")
    return insights

insights_avanzados = generar_insights_avanzados(df_clean, churn_column, cols_categoricas)
print("\nINSIGHTS PRINCIPALES:")
for i, ins in enumerate(insights_avanzados, 1):
    print(f"{i}. {ins}")


## 10. Informe ejecutivo final


In [ ]:
def generar_informe_ejecutivo(df, churn_col, insights, cols_numericas, cols_categoricas):
    print("TELECOM X - ANÁLISIS DE CHURN DE CLIENTES")
    print("=" * 50)
    print(f"Fecha del análisis: {datetime.now().strftime('%d/%m/%Y %H:%M')}")
    print(f"Analista: Sistema de Data Science")
    print(f"Dataset: {len(df)} clientes, {len(df.columns)} variables")
    print("\nRESUMEN EJECUTIVO:")
    print("-" * 25)
    for i, insight in enumerate(insights, 1):
        print(f"{i}. {insight}")
    print("\nPRINCIPALES HALLAZGOS:")
    print("-" * 30)
    hallazgos = []
    missing_percentage = (df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100
    if missing_percentage < 5:
        hallazgos.append("Excelente calidad de datos - menos del 5% de valores faltantes")
    elif missing_percentage < 15:
        hallazgos.append("Calidad de datos aceptable - requiere limpieza menor")
    else:
        hallazgos.append("Calidad de datos comprometida - requiere limpieza extensiva")
    if churn_col:
        unique_values = df[churn_col].nunique()
        if unique_values == 2:
            hallazgos.append("Variable de churn identificada correctamente - análisis binario posible")
        else:
            hallazgos.append("Variable de churn con múltiples categorías - requiere recodificación")
    if len(cols_numericas) >= 5:
        hallazgos.append("Dataset rico en variables numéricas - adecuado para modelos predictivos")
    categorical_diversity = sum(df[col].nunique() for col in cols_categoricas if df[col].nunique() <= 10) if cols_categoricas else 0
    if categorical_diversity >= 20:
        hallazgos.append("Múltiples dimensiones de segmentación disponibles")
    for i, h in enumerate(hallazgos, 1):
        print(f"{i}. {h}")
    print("\nRECOMENDACIONES ESTRATÉGICAS:")
    print("-" * 35)
    recomendaciones = [
        "Implementar modelo predictivo de churn (Random Forest o XGBoost)",
        "Desarrollar sistema de alertas tempranas para clientes en riesgo",
        "Crear campañas de retención personalizadas por segmentos",
        "Analizar el lifetime value para priorizar esfuerzos de retención",
        "Establecer programa de contact center proactivo para clientes de alto riesgo",
        "Diseñar ofertas y promociones basadas en patrones de comportamiento",
        "Implementar dashboard ejecutivo para monitoreo continuo de KPIs"
    ]
    for i, rec in enumerate(recomendaciones, 1):
        print(f"{i}. {rec}")
    print("\nPRÓXIMOS PASOS TÉCNICOS:")
    print("-" * 30)
    proximos_pasos = [
        "Feature engineering: crear variables derivadas y ratios",
        "Experimentación con algoritmos de ML: RF, XGB, Redes Neuronales",
        "Análisis de cohortes para entender comportamiento temporal",
        "Implementación de métricas de negocio: CLV, CAC, Payback period",
        "Desarrollo de visualizaciones interactivas con Plotly/Dash",
        "Deploy del modelo en producción (MLflow/Docker)",
        "A/B testing para validar estrategias de retención"
    ]
    for i, paso in enumerate(proximos_pasos, 1):
        print(f"{i}. {paso}")
    return hallazgos, recomendaciones, proximos_pasos

hallazgos_final, recomendaciones_final, pasos_final = generar_informe_ejecutivo(df_clean, churn_column, insights_avanzados, cols_numericas, cols_categoricas)


## 11. Métricas de negocio y ROI


In [ ]:
def calcular_impacto_negocio(df, churn_col):
    print("ANÁLISIS DE IMPACTO ECONÓMICO:")
    print("=" * 35)
    if churn_col and 'Cuentas_Diarias' in df.columns:
        total_clientes = len(df)
        if pd.api.types.is_numeric_dtype(df[churn_col]):
            clientes_churn = int((df[churn_col] == 1).sum())
            clientes_activos = int((df[churn_col] == 0).sum())
            tasa_churn = float(df[churn_col].mean())
            revenue_perdido = float(df.loc[df[churn_col] == 1, 'Cuentas_Diarias'].sum() * 365)
            revenue_activo = float(df.loc[df[churn_col] == 0, 'Cuentas_Diarias'].sum() * 365)
            revenue_promedio_churn = float(df.loc[df[churn_col] == 1, 'Cuentas_Diarias'].mean() * 365)
            revenue_promedio_activo = float(df.loc[df[churn_col] == 0, 'Cuentas_Diarias'].mean() * 365)
        else:
            vals = df[churn_col].dropna().unique()
            val_churn = vals[0] if len(vals) == 1 else ('Yes' if 'Yes' in vals else vals[0])
            clientes_churn = int((df[churn_col] == val_churn).sum())
            clientes_activos = int((df[churn_col] != val_churn).sum())
            tasa_churn = float((df[churn_col] == val_churn).mean())
            revenue_perdido = float(df.loc[df[churn_col] == val_churn, 'Cuentas_Diarias'].sum() * 365)
            revenue_activo = float(df.loc[df[churn_col] != val_churn, 'Cuentas_Diarias'].sum() * 365)
            revenue_promedio_churn = float(df.loc[df[churn_col] == val_churn, 'Cuentas_Diarias'].mean() * 365)
            revenue_promedio_activo = float(df.loc[df[churn_col] != val_churn, 'Cuentas_Diarias'].mean() * 365)
        total_revenue = revenue_perdido + revenue_activo
        print(f"Total de clientes: {total_clientes:,}")
        print(f"Clientes activos: {clientes_activos:,} ({(clientes_activos/total_clientes)*100:.1f}%)")
        print(f"Clientes perdidos: {clientes_churn:,} ({tasa_churn*100:.1f}%)")
        print(f"\nRevenue total anual: ${total_revenue:,.2f}")
        print(f"Revenue perdido anual: ${revenue_perdido:,.2f}")
        print(f"Revenue activo anual: ${revenue_activo:,.2f}")
        print("\nRevenue promedio por cliente:")
        print(f"  - Clientes que se van: ${revenue_promedio_churn:,.2f}/año")
        print(f"  - Clientes que se quedan: ${revenue_promedio_activo:,.2f}/año")
        if tasa_churn > 0 and not np.isnan(revenue_promedio_churn):
            reduccion_churn_25 = clientes_churn * 0.25
            revenue_recuperable_25 = reduccion_churn_25 * revenue_promedio_churn
            reduccion_churn_50 = clientes_churn * 0.50
            revenue_recuperable_50 = reduccion_churn_50 * revenue_promedio_churn
            print("\nPOTENCIAL DE MEJORA:")
            print(f"  - Reduciendo churn 25%: ${revenue_recuperable_25:,.2f} adicionales/año")
            print(f"  - Reduciendo churn 50%: ${revenue_recuperable_50:,.2f} adicionales/año")
            costo_retencion_cliente = revenue_promedio_churn * 0.15
            inversion_programa_25 = reduccion_churn_25 * costo_retencion_cliente
            if inversion_programa_25 > 0:
                roi_25 = (revenue_recuperable_25 - inversion_programa_25) / inversion_programa_25 * 100
                print("\nROI ESTIMADO (reducción 25% churn):")
                print(f"  - Inversión estimada: ${inversion_programa_25:,.2f}")
                print(f"  - ROI proyectado: {roi_25:.1f}%")
        return {
            'total_clientes': total_clientes,
            'clientes_churn': clientes_churn,
            'tasa_churn': tasa_churn,
            'revenue_perdido': revenue_perdido,
            'revenue_total': total_revenue
        }
    else:
        print("No se puede calcular impacto económico sin datos de churn y facturación")
        return None

metricas_negocio = calcular_impacto_negocio(df_clean, churn_column)


## 12. Exportación de resultados


In [ ]:
def exportar_resultados_completos(df, insights, hallazgos, recomendaciones, cols_numericas):
    try:
        df.to_csv('telecom_churn_data_clean.csv', index=False, encoding='utf-8')
        print("Dataset limpio exportado: telecom_churn_data_clean.csv")
        with open('telecom_churn_executive_report.txt', 'w', encoding='utf-8') as f:
            f.write("TELECOM X - REPORTE EJECUTIVO DE CHURN\n")
            f.write("="*50 + "\n\n")
            f.write(f"Fecha: {datetime.now().strftime('%d/%m/%Y %H:%M')}\n")
            f.write(f"Dataset: {len(df)} clientes analizados\n\n")
            f.write("INSIGHTS PRINCIPALES:\n")
            f.write("-"*25 + "\n")
            for i, insight in enumerate(insights, 1):
                f.write(f"{i}. {insight}\n")
            f.write("\nHALLAZGOS CLAVE:\n")
            f.write("-"*20 + "\n")
            for i, hallazgo in enumerate(hallazgos, 1):
                f.write(f"{i}. {hallazgo}\n")
            f.write("\nRECOMENDACIONES:\n")
            f.write("-"*20 + "\n")
            for i, rec in enumerate(recomendaciones, 1):
                f.write(f"{i}. {rec}\n")
        print("Reporte ejecutivo exportado: telecom_churn_executive_report.txt")
        if cols_numericas:
            estadisticas = df[cols_numericas].describe()
            estadisticas.to_csv('telecom_churn_descriptive_stats.csv', encoding='utf-8')
            print("Estadísticas descriptivas exportadas: telecom_churn_descriptive_stats.csv")
    except Exception as e:
        print(f"Error en exportación: {e}")

exportar_resultados_completos(df_clean, insights_avanzados, hallazgos_final, recomendaciones_final, cols_numericas)


## 13. Resumen final


In [ ]:
print("\n" + "="*60)
print("ANÁLISIS COMPLETADO EXITOSAMENTE")
print("="*60)
print(f"Clientes analizados: {len(df_clean):,}")
print(f"Variables procesadas: {len(df_clean.columns)}")
calidad = (1 - df_clean.isnull().sum().sum() / (len(df_clean) * len(df_clean.columns))) * 100
print(f"Calidad de datos: {calidad:.1f}%")
if churn_column:
    try:
        if pd.api.types.is_numeric_dtype(df_clean[churn_column]):
            tasa_final = df_clean[churn_column].mean()
        else:
            vals = df_clean[churn_column].dropna().unique()
            yes_alias = {'yes', 'sí', 'si', '1', 'true'}
            val_churn = None
            for v in vals:
                if str(v).strip().lower() in yes_alias:
                    val_churn = v
                    break
            if val_churn is None and len(vals) > 0:
                val_churn = vals[0]
            tasa_final = (df_clean[churn_column] == val_churn).mean()
        print(f"Tasa de churn identificada: {tasa_final:.1%}")
    except Exception as e:
        print(f"No se pudo calcular tasa final de churn: {e}")
if metricas_negocio:
    print(f"Revenue en riesgo (estimado): ${metricas_negocio['revenue_perdido']:,.2f}")
print(f"\nAnálisis finalizado: {datetime.now().strftime('%d/%m/%Y a las %H:%M')}")
print("Archivos generados:")
print("  - telecom_churn_data_clean.csv")
print("  - telecom_churn_executive_report.txt")
print("  - telecom_churn_descriptive_stats.csv")
print("\nProyecto listo para:")
print("  * Implementación de modelos de ML")
print("  * Desarrollo de estrategias de retención")
print("  * Presentación a stakeholders")
print("  * Despliegue en producción")


## 14. Manejo de error de carga (si falla la API)


In [ ]:
if df_raw is None:
    print("\nERROR CRÍTICO")
    print("No se pudieron cargar los datos desde la API.")
    print("Verificar:")
    print(" - Conexión a internet")
    print(" - Disponibilidad de la API")
    print(" - URL correcta del endpoint")
